In [ ]:
from PyPDF2 import PdfWriter, PdfReader
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, ContainerClient, BlobBlock, BlobClient
import shutil
import tempfile
import os

In [ ]:
filename = "test.pdf"
container_name = "good"

file_name = os.path.basename(filename)  
# Remove the file extension  
file_name_without_extension = os.path.splitext(file_name)[0] 
print(file_name_without_extension)

In [ ]:
account_url = "https://test.blob.core.windows.net/good"
credential = DefaultAzureCredential()
connection_string = ""
blob_ouptut_directory = file_name_without_extension + "/"

import glob, os


def delete_directory(directory_path):  
    try:  
        # Check if the directory exists  
        if not os.path.exists(directory_path):  
            print(f"Directory '{directory_path}' does not exist.")  
            return  
  
        # Check if the directory is empty  
        if len(os.listdir(directory_path)) == 0:  
            print(f"Directory '{directory_path}' is already empty.")  
        else:  
            # Delete all files in the directory  
            for file_name in os.listdir(directory_path):  
                file_path = os.path.join(directory_path, file_name)  
                if os.path.isfile(file_path):  
                    os.remove(file_path)  
                    print(f"Deleted file '{file_path}'")  
  
        # Delete the directory  
        os.rmdir(directory_path)  
        print(f"Deleted directory '{directory_path}'")  
  
    except Exception as e:  
        print(f"An exception occurred: {e}")  
        pass
        
        

local_directory = "./localcopy/"
output_directory = "./outputpdf/"
blob_name = filename


delete_directory(local_directory)
delete_directory(output_directory)

# Create the local directory if it doesn't exist  
if not os.path.exists(local_directory):  
    os.makedirs(local_directory)  
    
if not os.path.exists(output_directory):  
    os.makedirs(output_directory)  
       
  
# Create a BlobServiceClient object using the connection string  
blob_service_client = BlobServiceClient.from_connection_string(connection_string)  
  
# Get a reference to the container  
container_client = blob_service_client.get_container_client(container_name)  
  
# Download the blob to the local directory  
with open(os.path.join(local_directory, blob_name), "wb") as file:  
    blob_client = container_client.get_blob_client(blob_name)  
    file.write(blob_client.download_blob().readall())  
  
print("File downloaded successfully!")


local_file_path = os.path.join(local_directory, blob_name)
# #create unique pdfs
# print("create unique pdfs")

with open(local_file_path, "rb") as file:
    inputpdf = PdfReader(open(local_file_path, "rb"))
    print(len(inputpdf.pages))

    print('split up files')
    local_files = []
    blob_files = []
    for i in range(len(inputpdf.pages)):  
        output = PdfWriter()  
        output.add_page(inputpdf.pages[i])
        output_filename = f"{output_directory}{file_name_without_extension}_{i}.pdf"
        local_files.append(output_filename)
        blob_filename = f"{file_name_without_extension}/{file_name_without_extension}_{i}.pdf"
        blob_files.append(blob_filename)

        with open(output_filename, "wb") as output_stream:  
            output.write(output_stream)  
            print(output_stream.name)  


blob_service_client = BlobServiceClient.from_connection_string(connection_string) 
# Get a reference to the container  
container_client = blob_service_client.get_container_client(container_name)
    
for i in range(len(local_files)):
    splitfile = local_files[i]
    blobfile = blob_files[i]
    print('local split file = ' +  splitfile)
    print('blob file = ' + blobfile)
    
    with open(splitfile, "rb") as data:  
        # Check if the blob exists  
        if container_client.get_blob_client(blobfile).exists():  
            # Delete the existing blob  
            container_client.delete_blob(blobfile)  
        container_client.upload_blob(blobfile, data)

             


    

In [ ]:
local_directory = "./localcopy/"
output_directory = "./outputpdf/"

delete_directory(local_directory)
delete_directory(output_directory)